<a href="https://colab.research.google.com/github/sdsc-bw/MCXAI/blob/main/explain_covertype.ipynb#scrollTo=ce81b040" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explain a Model of Covertype Dataset

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, f1_score

import os
import sys
import zlib
import requests
from sklearn import datasets
from sklearn.utils import Bunch

In [2]:
from explainer import Explainer
import util

## Load and Prepare Dataset

We use the [sklearn covertype dataset](https://scikit-learn.org/0.16/datasets/covtype.html). The code for the dataset preparation and modelling based on [this](https://colab.research.google.com/github/grochmal/daml/blob/master/nb/ol-forest-cover.ipynb#scrollTo=pQnpMme2UUxq) notebook.

In [3]:
continuous = [
    'Elevation',
    'Aspect',
    'Slope',
    'HHydro',
    'VHydro',
    'Road',
    'Shade_9am',
    'Shade_Noon',
    'Shade_3pm',
    'Fire',
]
categorical = [
    'wild=1',  # Rawah Wilderness Area
    'wild=2',  # Neota Wilderness Area
    'wild=3',  # Comanche Peak Wilderness Area
    'wild=4',  # Cache la Poudre Wilderness Area
    'soil=1','soil=2','soil=3','soil=4','soil=5','soil=6','soil=7','soil=8','soil=9','soil=10',
    'soil=11','soil=12','soil=13','soil=14','soil=15','soil=16','soil=17','soil=18','soil=19','soil=20',
    'soil=21','soil=22','soil=23','soil=24','soil=25','soil=26','soil=27','soil=28','soil=29','soil=30',
    'soil=31','soil=32','soil=33','soil=34','soil=35','soil=36','soil=37','soil=38','soil=39','soil=40',
]
columns = continuous + categorical + ['label']
feature_names = continuous + categorical
target_names = ['Spruce/Fir', 'Lodgepole Pine', 'Ponderosa Pine',
                'Cottonwood/Willow', 'Aspen', 'Douglas-fir', 'Krummholz']

In [4]:
# Watch out: randomizes sample sequence!
def load_cover_type():
    cov_dir = 'uci_cover_type'
    data_dir = datasets.get_data_home()
    data_path = os.path.join(data_dir, cov_dir, 'covtype.data')
    descr_path = os.path.join(data_dir, cov_dir, 'covtype.info')
    cov_data_gz = 'https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz'
    cov_descr = 'https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.info'
    os.makedirs(os.path.join(data_dir, cov_dir), exist_ok=True)
    try:
        with open(descr_path, 'r') as f:
            descr = f.read()
    except IOError:
        print('Downloading file from', cov_descr, file=sys.stderr)
        r = requests.get(cov_descr)
        with open(descr_path, 'w') as f:
            f.write(r.text)
        descr = r.text
        r.close()
    try:
        data = pd.read_csv(data_path, delimiter=',', names=columns)
    except IOError:
        print('Downloading file from', cov_data_gz, file=sys.stderr)
        r = requests.get(cov_data_gz)
        cov_data = zlib.decompress(r.content, wbits=16+zlib.MAX_WBITS)  # obscure but works
        cov_data = cov_data.decode('utf8')
        with open(data_path, 'w') as f:
            f.write(cov_data)
        r.close()
        data = pd.read_csv(data_path, delimiter=',', names=columns)
    X = data[continuous + categorical].values
    y = data['label'].values - 1
    return Bunch(DESCR=descr,
                 data=X,
                 feature_names=columns[:-1],
                 feature_continuous=continuous,
                 feature_categorical=categorical,
                 target=y,
                 target_names=target_names)


covtype = load_cover_type()
print(covtype.DESCR[6923:8554])
print()
print(covtype.DESCR[12373:12713])

Name                                     Data Type    Measurement                       Description

Elevation                               quantitative    meters                       Elevation in meters
Aspect                                  quantitative    azimuth                      Aspect in degrees azimuth
Slope                                   quantitative    degrees                      Slope in degrees
Horizontal_Distance_To_Hydrology        quantitative    meters                       Horz Dist to nearest surface water features
Vertical_Distance_To_Hydrology          quantitative    meters                       Vert Dist to nearest surface water features
Horizontal_Distance_To_Roadways         quantitative    meters                       Horz Dist to nearest roadway
Hillshade_9am                           quantitative    0 to 255 index               Hillshade index at 9am, summer solstice
Hillshade_Noon                          quantitative    0 to 255 index              

In [5]:
df = pd.DataFrame(covtype.data, columns=covtype.feature_names)

In [6]:
# we only take two classes
X = covtype.data
y = covtype.target
X = X[(y == 1) | (y == 2)].copy()
y = y[(y == 1) | (y == 2)].copy()
y[y == 1] = 0
y[y == 2] = 1
df = pd.DataFrame(X, columns=covtype.feature_names)

In [7]:
# scaling of the continous features
sc = StandardScaler()
X_cont = sc.fit_transform(df[covtype.feature_continuous].values)
X_cat = df[covtype.feature_categorical].values
X = np.c_[X_cont, X_cat]
X.shape

(319055, 54)

## Train Model

In [8]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)

In [9]:
model = make_pipeline(
    PCA(n_components=10),
    SGDClassifier(loss='log', penalty='l2', max_iter=500, alpha=0.01, tol=0.01,
                  learning_rate='constant', eta0=0.001))
param_grid = {
    'sgdclassifier__alpha': [0.01, 0.1],
    'sgdclassifier__tol': [0.01, 0.1],
}
grid = GridSearchCV(model, param_grid, cv=5)
grid.fit(xtrain, ytrain)
grid.best_score_, grid.best_estimator_

(0.9597483209259241,
 Pipeline(steps=[('pca', PCA(n_components=10)),
                 ('sgdclassifier',
                  SGDClassifier(alpha=0.01, eta0=0.001, learning_rate='constant',
                                loss='log', max_iter=500, tol=0.01))]))

In [10]:
names = ['Aspen', 'Douglas-fir']
yfit = grid.best_estimator_.predict(xtest)
print(classification_report(ytest, yfit, target_names=names))

              precision    recall  f1-score   support

       Aspen       0.97      0.99      0.98     56697
 Douglas-fir       0.89      0.72      0.80      7114

    accuracy                           0.96     63811
   macro avg       0.93      0.86      0.89     63811
weighted avg       0.96      0.96      0.96     63811



In [11]:
cov1, cov2, ycov1, ycov2 = train_test_split(X, y, test_size=0.9)

In [12]:
xtrain1, xtest1, ytrain1, ytest1 = train_test_split(cov1, ycov1, test_size=0.2)
model = make_pipeline(
    PCA(n_components=10),
    SGDClassifier(loss='log', penalty='l2', max_iter=500, alpha=0.01, tol=0.01,
                  learning_rate='constant', eta0=0.001, warm_start=True))
model.fit(xtrain1, ytrain1)
yfit = model.predict(xtest1)
f1_score(ytest1, yfit)

0.7942811755361399

## Pick a Sample for Explanation

In [13]:
# here we take a sample form the second class
sample = X[(y == 1)][0]
target_label = y[(y == 1)][0]

In [14]:
sample

array([-1.23259517,  0.74326609,  0.34376537,  0.38476977,  0.81644083,
       -0.92178107, -0.89442147,  1.43608755,  1.46645668, -0.43760042,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ])

In [15]:
target_label

1

In [16]:
model.predict_proba([sample])

array([[0.50581211, 0.49418789]])

## Explain the Sample

The explainer allows to set different hyperparameters of the agent, game and tree. First it runs the classification game (tries to make thre prediction of the target label false). If the classification is already incorrect. It skips this game. After that it runs the misclassification game (tries to make thre prediction of the target label true). 

The games aren't always have a solution.

In [17]:
explainer = Explainer(sample, model.predict_proba, target_label, hide_value=0, max_episodes=100)

In [18]:
explainer.explain()

2021-10-29 15:39:01,836 - agent - INFO - XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2021-10-29 15:39:01,836 - agent - INFO - Episode:	0
2021-10-29 15:39:01,837 - agent - INFO - XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2021-10-29 15:39:01,882 - agent - INFO - XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2021-10-29 15:39:01,882 - agent - INFO - Episode:	10
2021-10-29 15:39:01,883 - agent - INFO - XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2021-10-29 15:39:01,915 - agent - INFO - XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2021-10-29 15:39:01,915 - agent - INFO - Episode:	20
2021-10-29 15:39:01,916 - agent - INFO - XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2021-10-29 15:39:01,946 - agent - INFO - XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2021-10-29 15:39:01,946 - agent - INFO - Episode:	30
2021-

In [19]:
ranks_1, ranks_2 = explainer.get_explanation()

Now the features are ranked by their importance, if the explainer was able to change the prediction (change the class with the highest probability). The lower the rank of the feature, the more important is this feature.

In [20]:
ranks_1

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [21]:
ranks_2

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

For better understanding, we kann also output the feature names with their corresponding ranks.

In [22]:
ranks_1_as_list, ranks_2_as_list = explainer.get_explanation_as_list(feature_names=feature_names)

In [23]:
ranks_1_as_list

[]

In [24]:
ranks_2_as_list

[('Shade_3pm', 1)]